# Setup

In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
# Standard library imports
from pydantic import BaseModel
import threading
import warnings
import random
import string
import queue
import json
import csv
import os

warnings.filterwarnings("ignore")
JWT_SECRET_API = !echo $(aws --profile "chat-prod_ro" secretsmanager get-secret-value --secret-id "arn:aws:secretsmanager:us-west-2:242659714806:secret:shared/cresta-server-jwt_secret-VDn5My" --query SecretString --output text) # type: ignore
os.environ["JWT_SECRET_API"] = json.loads(JWT_SECRET_API[0])["jwt-secret"]
os.environ["CONFIG_SERVICE_ADDR"] = "auth.chat-prod.internal.cresta.ai:443"
os.environ["CONFIG_USE_SECURE_CHANNEL"] = "true"

# Third party imports

from sentence_transformers import SentenceTransformer, util
from retry import retry

# Greyparrot imports
from greyparrot.llm.prompting import prompts as prompts_utils
from greyparrot.conversations.db import ConversationsDBConn
from greyparrot.multi_tenancy.v3_config import V3Config
from greyparrot.chats_common import PartialChat
from greyparrot.dataset_common import Dataset
from greyparrot.common import get_logger

logger = get_logger(__name__)

# Local imports
from llm_proxy_client import LLMProxyDevClient

In [41]:
embedder = SentenceTransformer("all-mpnet-base-v2")

In [42]:
customer_id = "brinks"
profile_id = "care-voice"
usecase_id = "care-voice"
language_code = "en-US"

In [43]:
def get_chats_with_ids(chat_ids: list[str]):
    customer_name = V3Config.short_name_from_ids(customer_id, profile_id)
    conv_db_conn = ConversationsDBConn.from_customer_name(customer_name)
    chats = conv_db_conn.get_detailed_chats(
        customer_id=customer_id,
        profile_id=profile_id,
        usecase_id=usecase_id,
        language_code=language_code,
        conversation_ids=chat_ids,
        is_dev_user=False,
    )
    return chats

In [44]:
LLM_ENGINE = "gpt-4o-mini"
CONCURRENCY = 10

In [45]:
@retry(tries=1, delay=60, backoff=2, logger=logger)
def chat_completion(**kwargs):
    return LLMProxyDevClient("openai").beta.chat.completions.parse(**kwargs)

# Discovery prompts

In [46]:
# TODO remove this after fixing speaker_role flips
flips = {"agent": "visitor", "visitor": "agent"}

In [47]:
def chat_to_prompt_text(chat: PartialChat, speakers_flipped: bool = False):
    return "\n".join([
        f"{string.capwords(prompts_utils.speaker_role_str_for_prompts(flips[m.speaker_role.value] if speakers_flipped else m.speaker_role.value))}: {m.text}"
        for m in chat.messages
    ])

In [48]:
SYSTEM_PROMPT_AGENT_WORKFLOW_DISCOVERY = """### Context and data description
You are a conversation analyst working for a Call Center.

You will be given 1 conversation at a time. Each conversation is between a Call Center Agent and a Customer. Your primary goal is to extract workflows of steps which the Agent takes in **the given conversation** to help resolve the Customer's needs related to a procut issue.

The primary use case of these workflows is to create a troubleshooting template to address similar customer needs in the future.

Each workflow should be a list of steps which the Agent needs to take.

For each workflow, return the product, issue, and a list of steps which the Agent needs to take.

Make sure the product is specific and not general.
Make sure the issue is specific and not general.
Make sure the steps are detailed.

**Important**: There could be more than 1 workflow in a single conversation. There could also be no workflows in a single conversation. The workflows will be used to create troubleshooting guides to address similar customer needs in the future."""

In [49]:
SYSTEM_PROMPT_VISITOR_WORKFLOW_DISCOVERY = """### Context and data description
You are a conversation analyst working for a Call Center.

You will be given 1 conversation at a time. Each conversation is between a Call Center Agent and a Customer. Your primary goal is to extract flows of steps which Customer takes in **the given conversation**.

The primary use case of this flow is to create a template to simulate similar customer scenarios.

The flow should be a list of steps which the Customer takes.

The schema of each flow should be as follows:
- **title**: title of the flow
- **steps**: a list of steps which Customer needs to follow

**Important**: There could be more than 1 flow in a single conversation. The flows will be used create templates to simulate similar customer scenarios."""

In [50]:
FLOW_PROMPTS = {
    "agent": SYSTEM_PROMPT_AGENT_WORKFLOW_DISCOVERY,
    "visitor": SYSTEM_PROMPT_VISITOR_WORKFLOW_DISCOVERY
}

In [51]:
class Flow(BaseModel):
    product: str
    issue: str
    steps: list[str]

class Flows(BaseModel):
    flows: list[Flow]

In [63]:
def discover_flow_in_chat(chat: PartialChat,
                          speaker_role: str,
                          speakers_flipped: bool = False,
                          llm_engine: str = LLM_ENGINE):
    logger.info(f"Discovering {speaker_role} flow in chat {chat.chat_name}")
    messages = [
        {
            "role": "system",
            "content": FLOW_PROMPTS[speaker_role]
        },
        {
            "role": "user",
            "content": chat_to_prompt_text(chat, speakers_flipped)
        },
    ]

    # Note: temperature=0.1 to allow for some exploration
    workflows = chat_completion(model=llm_engine,
                               messages=messages,
                               temperature=0.1,
                               response_format=Flows).choices[0].message.parsed.flows

    return workflows

In [64]:
def extract_flows_from_chats(chats: list[PartialChat], speaker_role: str,
                                concurrency: int = 10):
    lock = threading.Lock()
    indexes = queue.Queue()

    workflows = {}
    for idx in range(len(chats)):
        indexes.put(idx)

    def workflow_labeler_worker():
        while True:
            try:
                idx = indexes.get(block=False)
            except queue.Empty:
                return
            chat = chats[idx]
            try:
                extracted_workflows = discover_flow_in_chat(chat, speaker_role)
                with lock:
                    workflows[str(chat)] = extracted_workflows
                    if len(workflows) % 10 == 0:
                        print(f"Workflows from {len(workflows)} chats extracted!")
            except Exception as e:
                logger.warning(e, str(chat))
            indexes.task_done()

    logger.info(
        f"Starting processing {len(chats)} chats with {concurrency} workers")
    workers = [
        threading.Thread(target=workflow_labeler_worker)
        for _ in range(concurrency)
    ]
    for worker in workers:
        worker.start()
    for worker in workers:
        worker.join()
    logger.info(f"Finished processing all {len(chats)} chats")

    return workflows

In [65]:
test_chat = get_chats_with_ids(["0843c54c-6487-45ce-946a-cc6257484f54"])[0]
workflows = discover_flow_in_chat(test_chat, "agent", speakers_flipped=True)

{"message": "use case id not set, fallback to use profile id care-voice", "name": "greyparrot.common.customers", "asctime": "2025-02-13 18:42:20", "levelname": "WARNING", "filename": "customers.py", "funcName": "__init__", "threadName": "MainThread", "status": "WARNING"}
{"message": "use case id not set, fallback to use profile id care-voice", "name": "greyparrot.common.customers", "asctime": "2025-02-13 18:42:22", "levelname": "WARNING", "filename": "customers.py", "funcName": "__init__", "threadName": "MainThread", "status": "WARNING"}


cmd: cresta-cli connstring -i voice-prod voice-prod brinks-care-voice-FKrGHU -r
cmd: cresta-cli connstring -i voice-prod voice-prod brinks-care-voice -r


{"message": "Discovering agent flow in chat 0843c54c-6487-45ce-946a-cc6257484f54", "name": "__main__", "asctime": "2025-02-13 18:42:25", "levelname": "INFO", "filename": "<ipython-input-63-64321d38dd3d>", "funcName": "discover_flow_in_chat", "threadName": "MainThread", "status": "INFO"}


In [71]:
workflows.__len__()

1

In [80]:
print(f"Found {len(workflows)} workflows\n")
for i, flow in enumerate(workflows):
    print(f"[Agent Workflow #{i + 1}]")
    print(f"Product: {flow.product}")
    print(f"Issue: {flow.issue}")
    print("Steps:", end="\n* ")
    print("\n* ".join(flow.steps))
    if i < len(workflows) - 1:
        print("-" * 100)

Found 1 workflows

[Agent Workflow #1]
Product: Brinks Home Front Door Camera
Issue: Front door camera not responding
Steps:
* Verify customer account details including phone number and address.
* Ask the customer to check the LED light on the front door camera.
* Instruct the customer to unplug the power supply of the front door camera and wait for 30 seconds to 1 minute.
* Ask the customer to reconnect the power supply to the front door camera.
* Check the LED light status again after reconnecting the power supply.
* If no light is present, instruct the customer to test the power outlet with another device (like a phone charger) to ensure it is supplying power.
* Confirm that the outlet is working by verifying that the phone is charging.
* Determine that the power supply for the front door camera is faulty if the outlet is working and the camera still shows no light.
* Provide the customer with the contact number for the technician who can fix the camera.


# Relevant chats (from KA-QE trainset)

In [81]:
qe_dataset = Dataset.pull_from_repo(
    "brinks-care-voice/hf:082720241521675544.train")
len(qe_dataset)

{"message": "Pulling dataset brinks-care-voice/hf:082720241521675544.train from repo", "name": "dataset-common", "asctime": "2025-02-13 18:49:18", "levelname": "INFO", "filename": "common.py", "funcName": "pull_from_repo", "threadName": "MainThread", "status": "INFO"}
{"message": "use case id not set, fallback to use profile id care-voice", "name": "greyparrot.common.customers", "asctime": "2025-02-13 18:49:18", "levelname": "WARNING", "filename": "customers.py", "funcName": "__init__", "threadName": "MainThread", "status": "WARNING"}


RuntimeError: Failed to copy from s3://cresta-data-pci/brinks-care-voice/hf/082720241521675544.train/class.pkl to /tmp/ymwj7sxw: b'fatal error: An error occurred (403) when calling the HeadObject operation: Forbidden\n'

In [24]:
qe_dataset[0]

(a37ae6b3-d890-44c8-add6-3efb94e36244[:14],
 'How to stop double drafting by ADT and Brinks for alarm system?')

In [25]:
chat_ids = list(set([pc.chat_name for pc, _ in qe_dataset]))
len(chat_ids)

2369

In [26]:
queries_to_chat = {query: pc.chat_name for pc, query in qe_dataset}

In [38]:
queries = list(queries_to_chat.keys())
len(queries)

5989

In [28]:
source_embeddings = embedder.encode(queries,
                                    convert_to_tensor=True,
                                    show_progress_bar=False)
len(source_embeddings)

5989

In [36]:
with open("Brinks Evaluation v.20240806_ - Response Evaluation (internal).csv") as f:
    reader = csv.DictReader(f)
    data = list(reader)
    customer_queries = [entry["Question"] for entry in data]
len(customer_queries)

51

In [40]:
matched_queries = []
for q in customer_queries:
    target_embeddings = embedder.encode(q,
                                        convert_to_tensor=True,
                                        show_progress_bar=False)
    scores = util.cos_sim(target_embeddings,
                          source_embeddings).cpu().tolist()[0]
    matches = [(idx, score) for idx, score in enumerate(scores) if score > 0.7]
    if matches:
        matched_queries.extend([queries[i] for i, s in matches])

In [41]:
len(matched_queries)

75

In [43]:
matched_chats = [queries_to_chat[q] for q in matched_queries]
len(matched_chats)

75

In [46]:
num_chats = 1000
sampled_chat_ids = list(set(random.sample(chat_ids, num_chats) + matched_chats))
print(len(sampled_chat_ids))
sampled_chats = get_chats_with_ids(sampled_chat_ids)
len(sampled_chats)

1042


{"message": "use case id not set, fallback to use profile id care-voice", "name": "greyparrot.common.customers", "asctime": "2024-08-27 23:50:32", "levelname": "WARNING", "filename": "customers.py", "funcName": "__init__", "threadName": "MainThread", "status": "WARNING"}
{"message": "use case id not set, fallback to use profile id care-voice", "name": "greyparrot.common.customers", "asctime": "2024-08-27 23:50:34", "levelname": "WARNING", "filename": "customers.py", "funcName": "__init__", "threadName": "MainThread", "status": "WARNING"}


cmd: cresta-cli connstring -i voice-prod voice-prod brinks-care-voice-FKrGHU -r
cmd: cresta-cli connstring -i voice-prod voice-prod brinks-care-voice -r


1042

# Agent Workflow Discovery

In [ ]:
workflows = extract_flows_from_chats(sampled_chats, "agent")

{"message": "Starting processing 1042 chats with 10 workers", "name": "__main__", "asctime": "2024-08-27 23:51:13", "levelname": "INFO", "filename": "2872213784.py", "funcName": "extract_workflow_from_chats", "threadName": "MainThread", "status": "INFO"}
{"message": "Discovering agent flow in chat 00001ce5-86b1-4a85-867e-a95d15538865", "name": "__main__", "asctime": "2024-08-27 23:51:13", "levelname": "INFO", "filename": "2363125687.py", "funcName": "discover_flow_in_chat", "threadName": "Thread-6 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering agent flow in chat 00059bd9-dfde-4710-be56-5b7baf1931e0", "name": "__main__", "asctime": "2024-08-27 23:51:13", "levelname": "INFO", "filename": "2363125687.py", "funcName": "discover_flow_in_chat", "threadName": "Thread-7 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering agent flow in chat 00523c8b-0189-4ac2-a34e-93abdeafa478", "name": "__main__", "asctime": "2024-08-27 23:51:13", "levelname": "INFO",

Workflows from 10 extracted!


{"message": "Discovering agent flow in chat 049c2d1b-f742-4e46-be75-28473a622425", "name": "__main__", "asctime": "2024-08-27 23:51:44", "levelname": "INFO", "filename": "2363125687.py", "funcName": "discover_flow_in_chat", "threadName": "Thread-11 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering agent flow in chat 057f639a-e82f-4bf2-95c1-03aafb420771", "name": "__main__", "asctime": "2024-08-27 23:51:45", "levelname": "INFO", "filename": "2363125687.py", "funcName": "discover_flow_in_chat", "threadName": "Thread-6 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering agent flow in chat 05e10909-2844-4710-beaa-bb85e5f6f18c", "name": "__main__", "asctime": "2024-08-27 23:51:46", "levelname": "INFO", "filename": "2363125687.py", "funcName": "discover_flow_in_chat", "threadName": "Thread-8 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering agent flow in chat 05fe2dce-903f-4d3f-a046-e697c9b4a674", "name": "__main__", "asctime": "20

Workflows from 20 extracted!


{"message": "Discovering agent flow in chat 06c18913-c1e4-4274-aed8-d9d9c0ce6528", "name": "__main__", "asctime": "2024-08-27 23:52:03", "levelname": "INFO", "filename": "2363125687.py", "funcName": "discover_flow_in_chat", "threadName": "Thread-8 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering agent flow in chat 06c7a277-4b56-40ef-be87-ffa95c1388d6", "name": "__main__", "asctime": "2024-08-27 23:52:06", "levelname": "INFO", "filename": "2363125687.py", "funcName": "discover_flow_in_chat", "threadName": "Thread-9 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering agent flow in chat 072722c1-d4b5-48e1-91be-c6311665f577", "name": "__main__", "asctime": "2024-08-27 23:52:06", "levelname": "INFO", "filename": "2363125687.py", "funcName": "discover_flow_in_chat", "threadName": "Thread-6 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering agent flow in chat 073fc9c1-6885-4f6d-b0b0-4ae4f6a2753b", "name": "__main__", "asctime": "202

...

{"message": "Discovering agent flow in chat c202c7ed-e36a-4002-af6e-f92a4f963097", "name": "__main__", "asctime": "2024-08-28 00:19:39", "levelname": "INFO", "filename": "2363125687.py", "funcName": "discover_flow_in_chat", "threadName": "Thread-6 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering agent flow in chat c214b170-d5c7-459f-a074-f289cd24dc15", "name": "__main__", "asctime": "2024-08-28 00:19:42", "levelname": "INFO", "filename": "2363125687.py", "funcName": "discover_flow_in_chat", "threadName": "Thread-8 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering agent flow in chat c2726aed-9c41-4f70-8929-9e5a1b0cd1b7", "name": "__main__", "asctime": "2024-08-28 00:19:43", "levelname": "INFO", "filename": "2363125687.py", "funcName": "discover_flow_in_chat", "threadName": "Thread-9 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering agent flow in chat c2bb6896-a430-41ec-85bd-42c22fa17874", "name": "__main__", "asctime": "202

Workflows from 830 extracted!


{"message": "Discovering agent flow in chat c46508ef-5312-40f6-b757-0ff2c24e40d2", "name": "__main__", "asctime": "2024-08-28 00:20:02", "levelname": "INFO", "filename": "2363125687.py", "funcName": "discover_flow_in_chat", "threadName": "Thread-13 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering agent flow in chat c4c219b2-81aa-4fae-9836-5725e36cc224", "name": "__main__", "asctime": "2024-08-28 00:20:02", "levelname": "INFO", "filename": "2363125687.py", "funcName": "discover_flow_in_chat", "threadName": "Thread-15 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering agent flow in chat c4c67546-8ef9-42d0-8926-f8b1c66dce7d", "name": "__main__", "asctime": "2024-08-28 00:20:02", "levelname": "INFO", "filename": "2363125687.py", "funcName": "discover_flow_in_chat", "threadName": "Thread-10 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering agent flow in chat c4cd95e3-1964-4270-a231-a9d95b4e5f99", "name": "__main__", "asctime": "

Workflows from 840 extracted!


{"message": "Discovering agent flow in chat c86e68d0-1127-416e-a190-76c0fcee3344", "name": "__main__", "asctime": "2024-08-28 00:20:12", "levelname": "INFO", "filename": "2363125687.py", "funcName": "discover_flow_in_chat", "threadName": "Thread-15 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering agent flow in chat c8ae38f5-f5ce-4183-be01-590b2accbd64", "name": "__main__", "asctime": "2024-08-28 00:20:17", "levelname": "INFO", "filename": "2363125687.py", "funcName": "discover_flow_in_chat", "threadName": "Thread-10 (workflow_labeler_worker)", "status": "INFO"}


In [52]:
len(workflows)

1042

In [42]:
for pc, ws in list(workflows.items())[:5]:
    print(f"\n\n<{pc}>")
    for idx, w in enumerate(ws):
        print("-" * 50)
        print(f"[Agent Workflow#{idx + 1}]")
        for k, v in w.items():
            if isinstance(v, list):
                val = "\n* ".join(v)
                print(f"{k.capitalize()}:\n* {val}")
            else:
                print(f"{k.capitalize()}: {v}")



<010c0ecf-e945-46c4-b11a-28bf1136225a>
--------------------------------------------------
[Agent Workflow#1]
Title: Setting Up New Security Codes for Employees
Steps:
* Greet the Customer: Start the conversation by thanking the customer for calling and introducing yourself.
* Identify the Customer's Need: Confirm the customer's request to set up new security codes for their employees.
* Clarify the Number of Codes Needed: Verify the number of new codes required by the customer.
* Collect Customer Verification Information: Ask for the customer's first and last name, request the verbal password associated with the account, and confirm the location address associated with the account to ensure accuracy.
* Gather Necessary Information: Inform the customer that you are collecting the required information for setting up new codes.
* Engage Technical Support: Notify the customer that assistance from the tech department will be needed to complete their request.
* Confirm Customer's Location:

In [ ]:
## start here

In [97]:
title_to_workflow, title_to_pc_id = {}, {}
for pc_id, ws in workflows.items():
    for idx, w in enumerate(ws):
        title_to_workflow[w["title"]] = w
        title_to_pc_id[w["title"]] = pc_id

In [98]:
len(title_to_workflow), len(title_to_pc_id)

(1822, 1822)

In [62]:
def group_semantically_similar_workflows(workflows: list[str], embedder: SentenceTransformer,
                                           semantic_threshold: int = 0.8):
    workflow_embeddings = embedder.encode(workflows, convert_to_tensor=True)
    workflow_groups, solo_workflows = [], []
    for idx, workflow in enumerate(workflows):
        existing_groups = [group for group in workflow_groups if idx in group]
        if existing_groups:
            assert len(
                existing_groups) == 1, "A workflow should only be in 1 group"
            continue

        print(f"Finding similar Workflows for Workflow#{idx}")

        scores = util.cos_sim(workflow_embeddings[idx:idx + 1],
                              workflow_embeddings).cpu().tolist()[0]
        matches = [
            i for i, score in enumerate(scores) if score > semantic_threshold
        ]
        filtered_matches = [i for i in matches if i != idx]
        if filtered_matches:
            outstanding_groups = []
            new_group = filtered_matches + [idx]
            for group in workflow_groups:
                if set(filtered_matches).intersection(set(group)):
                    new_group.extend(group)
                else:
                    outstanding_groups.append(group)
            workflow_groups = outstanding_groups + [list(set(new_group))]
        else:
            solo_workflows.append(idx)

    print(f"Found {len(solo_workflows)} # of Solo Workflows..")
    print(f"Found {len(workflow_groups)} # Groups of Workflows..")

    return [workflows[idx] for idx in solo_workflows
           ], [[workflows[idx] for idx in group] for group in workflow_groups]

In [64]:
solo_workflows, grouped_workflows = group_semantically_similar_workflows(
    list(title_to_workflow.keys()), embedder)

Finding similar Workflows for Workflow#0
Finding similar Workflows for Workflow#1
Finding similar Workflows for Workflow#2
Finding similar Workflows for Workflow#3
Finding similar Workflows for Workflow#5
Finding similar Workflows for Workflow#6
Finding similar Workflows for Workflow#7
Finding similar Workflows for Workflow#8
Finding similar Workflows for Workflow#9
Finding similar Workflows for Workflow#10
Finding similar Workflows for Workflow#11
Finding similar Workflows for Workflow#12
Finding similar Workflows for Workflow#14
Finding similar Workflows for Workflow#15
Finding similar Workflows for Workflow#16
Finding similar Workflows for Workflow#17
Finding similar Workflows for Workflow#18
Finding similar Workflows for Workflow#19
Finding similar Workflows for Workflow#20
Finding similar Workflows for Workflow#21
Finding similar Workflows for Workflow#22
Finding similar Workflows for Workflow#24
Finding similar Workflows for Workflow#25
Finding similar Workflows for Workflow#26
F

In [65]:
for each in grouped_workflows:
    print("-" * 50)
    print(len(each))
    for e in each:
        print(e)

--------------------------------------------------
2
Resolving Smoke Detector and Window Sensor Alerts
Resolving Smoke Detector Offline and Failure Alerts
--------------------------------------------------
4
Troubleshooting and Scheduling Technician for Camera Issue
Resolving Camera Connectivity Issue and Scheduling Technician Visit
Scheduling a Technician Visit for Camera Repair
Scheduling a Technician Appointment for Camera and System Setup
--------------------------------------------------
4
New Owner Account Takeover Process
New Owner Takeover Process
Transfer Process for New Owner Takeover
Internal Transfer Process for New Owner Takeover
--------------------------------------------------
2
Offering Alternative Customer Support Options
Offering Customer Support and Self-Help Options
--------------------------------------------------
2
Checking Eligibility for Equipment Upgrade
Product Upgrade Eligibility Check
--------------------------------------------------
3
Checking Contract E

In [67]:
unique_workflow_titles = solo_workflows + [random.choice(w) for w in grouped_workflows]
unique_workflows = [title_to_workflow[w] for w in unique_workflow_titles]
len(unique_workflows)

919

In [83]:
filtered_unique_workflows = [w for w in unique_workflows if len(w["steps"]) > 3] # skip too small workflows
len(filtered_unique_workflows)

905

In [84]:
filtered_unique_workflows[0]

{'title': 'Verifying Customer Information and Addressing Contract Expiry Inquiry',
 'steps': ['Greet the customer and introduce yourself.',
  "Ask for the customer's name for personalization.",
  'Correctly address the customer upon clarification (e.g., Mr., Mrs., Miss).',
  'For verification purposes, request the customer to verify their address and the account code word.',
  'If the customer has multiple accounts or addresses, ask for details of each one sequentially.',
  'Confirm the ZIP code and code word for each account mentioned.',
  "Address the primary reason for the customer's call, such as contract expiration inquiry.",
  'If necessary, search for additional accounts to ensure all customer concerns are addressed.',
  'Confirm if the customer is experiencing any issues with the system, such as login problems.',
  'Address any immediate concerns the customer has, such as username and password issues.',
  'Provide the customer with the specific information they requested, such 

In [85]:
for w in filtered_unique_workflows[:10]:
    print(w["title"])

Verifying Customer Information and Addressing Contract Expiry Inquiry
Resolving App Login Issues and Understanding Device Settings Migration
Navigating to the Correct Website for Account Management
Discussing Account Renewal Options and Promotions
Expanding Services to Additional Properties
Call Transfer Process
Enabling Test Mode for Alarm System Maintenance
Troubleshooting a Front Door Camera That Is Not Recording
Resolving Camera Issues for Customers with Pending Installation Status
Workaround for Creating a Ticket When Unable to Select an Option Due to System Restrictions


In [86]:
for w in filtered_unique_workflows:
    print("-" * 100)
    print("\n\n")
    for k, v in w.items():
        if isinstance(v, list):
            val = "\n* ".join(v)
            print(f"{k.capitalize()}:\n* {val}")
        else:
            print(f"{k.capitalize()}: {v}")    

----------------------------------------------------------------------------------------------------



Title: Verifying Customer Information and Addressing Contract Expiry Inquiry
Steps:
* Greet the customer and introduce yourself.
* Ask for the customer's name for personalization.
* Correctly address the customer upon clarification (e.g., Mr., Mrs., Miss).
* For verification purposes, request the customer to verify their address and the account code word.
* If the customer has multiple accounts or addresses, ask for details of each one sequentially.
* Confirm the ZIP code and code word for each account mentioned.
* Address the primary reason for the customer's call, such as contract expiration inquiry.
* If necessary, search for additional accounts to ensure all customer concerns are addressed.
* Confirm if the customer is experiencing any issues with the system, such as login problems.
* Address any immediate concerns the customer has, such as username and password issues.
* Provide

In [87]:
[w["title"] for w in filtered_unique_workflows]

['Verifying Customer Information and Addressing Contract Expiry Inquiry',
 'Resolving App Login Issues and Understanding Device Settings Migration',
 'Navigating to the Correct Website for Account Management',
 'Discussing Account Renewal Options and Promotions',
 'Expanding Services to Additional Properties',
 'Call Transfer Process',
 'Enabling Test Mode for Alarm System Maintenance',
 'Troubleshooting a Front Door Camera That Is Not Recording',
 'Resolving Camera Issues for Customers with Pending Installation Status',
 'Workaround for Creating a Ticket When Unable to Select an Option Due to System Restrictions',
 'Service Transfer Process',
 'Troubleshooting Network Connection for IQ Panel 2',
 'Account Cancellation Process for Assisted Living Situations',
 'Effective Support Call Handling',
 'Exploring Contract Renewal and Warranty Options',
 'Managing Tamper Alerts Post Battery Replacement',
 'Resolving Missing Equipment Shipment',
 'Addressing Early Termination Inquiry',
 'Proces

In [129]:
len(filtered_unique_workflows)

905